In [1]:
# imports
import shutil
from pathlib import Path

from ipywidgets import IntProgress, Label, VBox
from IPython.display import display

In [3]:
# set network path and get list of directories to process
data_directory = Path('/Volumes/fluffy/0_JeremysWorkingSpace/2019-10/board-of-regents_toOCR')
input_directory = data_directory.joinpath('1.toOCR')
output_directory = data_directory.joinpath('2.OCRed')

directories_to_convert_to_pdf_paths_list = sorted([x for x in input_directory.iterdir() if x.is_dir()])

print(f'{len(directories_to_convert_to_pdf_paths_list)} directories to process')

0 directories to process


In [4]:
!open {str(data_directory)}

In [6]:
# functions

def get_tif_paths_list(directory_path, scantailor=False):
    
    # if directory_path not a Path-like object then make it one
    if not isinstance(directory_path, Path):
        directory_path = Path(directory_path)
        
    # add 'out' path to directory if processed with ScanTailor
    if scantailor:
        directory_path = Path(directory_path).joinpath('out')
    
    # get paths to *.tif to process
    tif_paths_list = sorted(directory_path.glob('*.tif'))
    
    if len(tif_paths_list) == 0:
        tif_paths_list = sorted(directory_path.glob('*.tiff'))
    
    # only keep *.tif that DO NOT start with '.' to exclude macOS index files
    tif_paths_list = [x for x in tif_paths_list if not str(x.name).startswith('.')]
    
    return tif_paths_list


def create_temporary_pdf_directory(directory_path):
    
    # if directory_path not a Path-like object then make it one
    if not isinstance(directory_path, Path):
        directory_path = Path(directory_path)
        
    temporary_pdf_directory_path = Path.home().joinpath(f'Desktop/_temporary_pdfs_{directory_path.name}')
    
    try:  # creating the temporary directory
        temporary_pdf_directory_path.mkdir()
    except FileExistsError:  # delete the directory & all contents then create it
        shutil.rmtree(temporary_pdf_directory_path)
        temporary_pdf_directory_path.mkdir()
        
    return temporary_pdf_directory_path


def batch_ocr_tifs_into_pdfs(tif_paths_list, temporary_pdf_directory_path):
    
    number_of_tifs = len(tif_paths_list)
    number_of_pdfs = 0
    
    # progress bar
    progress_label = Label('OCR *.tif')
    progress_bar = IntProgress(min=0, max=number_of_tifs)
    progress_widget = VBox([progress_label, progress_bar])
    display(progress_widget)
    
    for tif_path in tif_paths_list:
        
        label = f'OCRing {tif_path.name}'
        progress_label.value = label

        temporary_pdf_output_path = temporary_pdf_directory_path.joinpath(tif_path.stem)

        # ocr *.tif with tesseract
        # NOTE: output SHOULD NOT have extension on end!
        !tesseract "{str(tif_path)}" "{str(temporary_pdf_output_path)}" pdf 2>/dev/null

        # Add the suffix .pdf that tesseract didn't want
        temporary_pdf_output_path = temporary_pdf_directory_path.joinpath(f'{tif_path.stem}.pdf') 

        if temporary_pdf_output_path.is_file():
            number_of_pdfs += 1
            progress_bar.value = number_of_pdfs
        
    if number_of_pdfs != number_of_tifs:
        print(f'ERROR - ERROR - ERROR')
        print(f'# of *.tif: {number_of_tifs}')
        print(f'# of PDFs: {number_of_pdfs}')
    else:
        print(f'{number_of_pdfs} PDFs created and saved in {temporary_pdf_directory_path}')
    
    
def concatenate_pdfs(temporary_pdf_directory_path):

    # strip '_temporary_pdfs_' from beginning of directory path to get combined PDF name
    combined_pdf_name = f"{str(temporary_pdf_directory_path).split('_temporary_pdfs_')[1]}.pdf"

    combined_pdf_output_path = temporary_pdf_directory_path.joinpath(combined_pdf_name)

    # concatenate with pdftk
    !pdftk "{str(temporary_pdf_directory_path)}"/*.pdf cat output "{str(combined_pdf_output_path)}"

    if combined_pdf_output_path.is_file() and combined_pdf_output_path.stat().st_size > 0:
        print(f'{combined_pdf_output_path} created with size {round((combined_pdf_output_path.stat().st_size/1024/1024/1024), 2)} GB')

    return combined_pdf_output_path

In [9]:
for dir_path in directories_to_convert_to_pdf_paths_list:
    
    # get list of *.tif to process
    tif_paths_list = get_tif_paths_list(dir_path)
    
    number_of_tifs = len(tif_paths_list)
    print(f'{number_of_tifs} *.tif to process in {dir_path}')
    
    # create temp directory
    temp_pdf_dir_path = create_temporary_pdf_directory(dir_path)
    
    # OCR tifs into PDFs with progress bar
    batch_ocr_tifs_into_pdfs(tif_paths_list, temp_pdf_dir_path)
    
    # concatenate PDFs into single file
    combined_pdf_path = concatenate_pdfs(temp_pdf_dir_path)
    
    # copy combined PDF to copy directory
    output_pdf_path = output_directory.joinpath(combined_pdf_path.name)
    shutil.copy(combined_pdf_path, output_pdf_path)
    
    if output_pdf_path.is_file():  # move processed dir_path into output_directory
        renamed_dir_path = output_directory.joinpath(dir_path.name)
        dir_path.rename(renamed_dir_path)

    print('')
    print('*****')    
    print(f'{dir_path.name} is now {renamed_dir_path}')
    print('*****')
    print('')

514 *.tif to process in /Volumes/fluffy/0_JeremysWorkingSpace/2019-10/board-of-regents_toOCR/1.toOCR/R_ BOT_1977-09-23_EX1-26


514 PDFs created and saved in /Users/dlisla/Desktop/_temporary_pdfs_R_ BOT_1977-09-23_EX1-26
/Users/dlisla/Desktop/_temporary_pdfs_R_ BOT_1977-09-23_EX1-26/R_ BOT_1977-09-23_EX1-26.pdf created with size 0.12 GB

*****
R_ BOT_1977-09-23_EX1-26 is now /Volumes/fluffy/0_JeremysWorkingSpace/2019-10/board-of-regents_toOCR/2.OCRed/R_ BOT_1977-09-23_EX1-26
*****

46 *.tif to process in /Volumes/fluffy/0_JeremysWorkingSpace/2019-10/board-of-regents_toOCR/1.toOCR/R_ BOT_1977-10-29_EX1-7


46 PDFs created and saved in /Users/dlisla/Desktop/_temporary_pdfs_R_ BOT_1977-10-29_EX1-7
/Users/dlisla/Desktop/_temporary_pdfs_R_ BOT_1977-10-29_EX1-7/R_ BOT_1977-10-29_EX1-7.pdf created with size 0.01 GB

*****
R_ BOT_1977-10-29_EX1-7 is now /Volumes/fluffy/0_JeremysWorkingSpace/2019-10/board-of-regents_toOCR/2.OCRed/R_ BOT_1977-10-29_EX1-7
*****

17 *.tif to process in /Volumes/fluffy/0_JeremysWorkingSpace/2019-10/board-of-regents_toOCR/1.toOCR/R_ BOT_1977-11-26_EX1


17 PDFs created and saved in /Users/dlisla/Desktop/_temporary_pdfs_R_ BOT_1977-11-26_EX1
/Users/dlisla/Desktop/_temporary_pdfs_R_ BOT_1977-11-26_EX1/R_ BOT_1977-11-26_EX1.pdf created with size 0.0 GB

*****
R_ BOT_1977-11-26_EX1 is now /Volumes/fluffy/0_JeremysWorkingSpace/2019-10/board-of-regents_toOCR/2.OCRed/R_ BOT_1977-11-26_EX1
*****

833 *.tif to process in /Volumes/fluffy/0_JeremysWorkingSpace/2019-10/board-of-regents_toOCR/1.toOCR/R_ BOT_1978-02-24_EX1-38


833 PDFs created and saved in /Users/dlisla/Desktop/_temporary_pdfs_R_ BOT_1978-02-24_EX1-38
/Users/dlisla/Desktop/_temporary_pdfs_R_ BOT_1978-02-24_EX1-38/R_ BOT_1978-02-24_EX1-38.pdf created with size 0.15 GB

*****
R_ BOT_1978-02-24_EX1-38 is now /Volumes/fluffy/0_JeremysWorkingSpace/2019-10/board-of-regents_toOCR/2.OCRed/R_ BOT_1978-02-24_EX1-38
*****

11 *.tif to process in /Volumes/fluffy/0_JeremysWorkingSpace/2019-10/board-of-regents_toOCR/1.toOCR/R_ BOT_1978-05-31_EX1-5


...

62 PDFs created and saved in /Users/dlisla/Desktop/_temporary_pdfs_R_ BOT_1982-03-10_EX1-5
/Users/dlisla/Desktop/_temporary_pdfs_R_ BOT_1982-03-10_EX1-5/R_ BOT_1982-03-10_EX1-5.pdf created with size 0.01 GB

*****
R_ BOT_1982-03-10_EX1-5 is now /Volumes/fluffy/0_JeremysWorkingSpace/2019-10/board-of-regents_toOCR/2.OCRed/R_ BOT_1982-03-10_EX1-5
*****

44 *.tif to process in /Volumes/fluffy/0_JeremysWorkingSpace/2019-10/board-of-regents_toOCR/1.toOCR/R_ BOT_1982-04-23_EX1-8


44 PDFs created and saved in /Users/dlisla/Desktop/_temporary_pdfs_R_ BOT_1982-04-23_EX1-8
/Users/dlisla/Desktop/_temporary_pdfs_R_ BOT_1982-04-23_EX1-8/R_ BOT_1982-04-23_EX1-8.pdf created with size 0.01 GB

*****
R_ BOT_1982-04-23_EX1-8 is now /Volumes/fluffy/0_JeremysWorkingSpace/2019-10/board-of-regents_toOCR/2.OCRed/R_ BOT_1982-04-23_EX1-8
*****

917 *.tif to process in /Volumes/fluffy/0_JeremysWorkingSpace/2019-10/board-of-regents_toOCR/1.toOCR/R_ BOT_1982-06-17_EX1-68


917 PDFs created and saved in /Users/dlisla/Desktop/_temporary_pdfs_R_ BOT_1982-06-17_EX1-68
/Users/dlisla/Desktop/_temporary_pdfs_R_ BOT_1982-06-17_EX1-68/R_ BOT_1982-06-17_EX1-68.pdf created with size 0.14 GB

*****
R_ BOT_1982-06-17_EX1-68 is now /Volumes/fluffy/0_JeremysWorkingSpace/2019-10/board-of-regents_toOCR/2.OCRed/R_ BOT_1982-06-17_EX1-68
*****

44 *.tif to process in /Volumes/fluffy/0_JeremysWorkingSpace/2019-10/board-of-regents_toOCR/1.toOCR/R_ BOT_1982-08-14_EX1-6


44 PDFs created and saved in /Users/dlisla/Desktop/_temporary_pdfs_R_ BOT_1982-08-14_EX1-6
/Users/dlisla/Desktop/_temporary_pdfs_R_ BOT_1982-08-14_EX1-6/R_ BOT_1982-08-14_EX1-6.pdf created with size 0.01 GB

*****
R_ BOT_1982-08-14_EX1-6 is now /Volumes/fluffy/0_JeremysWorkingSpace/2019-10/board-of-regents_toOCR/2.OCRed/R_ BOT_1982-08-14_EX1-6
*****

604 *.tif to process in /Volumes/fluffy/0_JeremysWorkingSpace/2019-10/board-of-regents_toOCR/1.toOCR/R_ BOT_1982-10-15_EX1-38


604 PDFs created and saved in /Users/dlisla/Desktop/_temporary_pdfs_R_ BOT_1982-10-15_EX1-38
/Users/dlisla/Desktop/_temporary_pdfs_R_ BOT_1982-10-15_EX1-38/R_ BOT_1982-10-15_EX1-38.pdf created with size 0.14 GB

*****
R_ BOT_1982-10-15_EX1-38 is now /Volumes/fluffy/0_JeremysWorkingSpace/2019-10/board-of-regents_toOCR/2.OCRed/R_ BOT_1982-10-15_EX1-38
*****



In [40]:
# open output_directory_path for visual Quality Assurance
!open {str(output_directory)}

In [4]:
# delete all '_temporary_pdfs_*' directories on the desktop
all_temp_dir_paths = sorted(Path('/Users/dlisla/Desktop/').glob('_temporary_pdfs_*'))
number_of_dir_paths = len(all_temp_dir_paths)
for temp_dir_path in all_temp_dir_paths:
    shutil.rmtree(temp_dir_path)

all_temp_dir_paths = sorted(Path('/Users/dlisla/Desktop/').glob('_temporary_pdfs_*'))
number_of_deleted_dir_paths = number_of_dir_paths - len(all_temp_dir_paths)

print(f'{number_of_deleted_dir_paths} _temporary_pdf_* directories deleted')

121 _temporary_pdf_* directories deleted
